# Bài thực hành số 2.1

Chào mừng đến với bài thực hành số 2.1, trong bài thực hành này ta sẽ học cách xử lý data từ data thô để chuẩn bị cho quá trình train và đánh giá model.

## Hướng dẫn

Dưới đây là hướng dẫn chi tiết các bước để bạn có thể hiểu được quá trình xử lý data


### Cài đặt thư viện cần thiết

In [ ]:
import os
import cv2
import numpy as np

### Điều chỉnh kích thước ảnh

In [ ]:
# đường dẫn tới ảnh cần đọc
img = cv2.imread([...])
original_height, original_width = img.shape[:2]
print(f"Original size: {original_width}x{original_height}")

In [ ]:
# Hiển thị hình ảnh ban đầu
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### Lời giải bài tập 1

In [ ]:
def resize_image(image, target_width=320, target_height=240):
    # Lấy kích thước ảnh
    original_height, original_width = image.shape[:2]

    # Tính toán tỷ lệ
    target_ratio = target_width / target_height
    original_ratio = original_width / original_height

    if original_ratio > target_ratio:
        # Ảnh rộng hơn, cắt chiều ngang
        new_width = int(original_height * target_ratio)
        new_height = original_height
        x_offset = (original_width - new_width) // 2
        image = image[:, x_offset:x_offset + new_width]
    else:
        # Ảnh cao hơn, cắt chiều dọc
        new_height = int(original_width / target_ratio)
        new_width = original_width
        y_offset = (original_height - new_height) // 2
        image = image[y_offset:y_offset + new_height, :]

    # Resize ảnh để phù hợp với kích thước mục tiêu
    resized_img = cv2.resize(image, (target_width, target_height), interpolation=cv2.INTER_AREA)

    return resized_img


In [ ]:
resized_image = resize_image(img, 320, 240)

#### Kết quả

In [ ]:
# Hiển thị hình ảnh sau khi đổi kích thước
cv2.imshow("test", resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Tăng cường dữ liệu

#### Lời giải bài tập 2

In [ ]:
# Điều chỉnh độ sáng
def adjust_brightness(image, value):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v, value)
    v = np.clip(v, 0, 255)
    final_hsv = cv2.merge((h, s, v))
    img_bright = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img_bright

In [ ]:
# Lật ảnh ngang
def flip_horizontal(image):
    return cv2.flip(image, 1)

In [ ]:
# Làm mờ ảnh
def blur_image(image, ksize=5):
    return cv2.GaussianBlur(image, (ksize, ksize), 0)

In [ ]:
# Thêm nhiễu
def add_noise(image):
    row, col, ch = image.shape
    mean = 0
    sigma = 25
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

In [ ]:
# Tăng sáng
bright_img = adjust_brightness(resized_image, 50)

# Giảm sáng
dark_img = adjust_brightness(resized_image, -50)

# Lật ảnh ngang
flipped_img = flip_horizontal(resized_image)

# Làm mờ
blurred_img = blur_image(resized_image)

# Thêm nhiễu
noisy_img = add_noise(resized_image)

#### Kết quả

In [ ]:
# Hiển thị ảnh sau xử lý

# Tên ảnh
titles = ["Resized image", "Bright image", "Dark img", "Flipped image", "Blurred image", "Noisy image"]

# Giả sử các ảnh đã được resize về cùng kích thước
height, width = resized_image.shape[:2]

# Hàm để thêm tên dưới mỗi ảnh
def add_title(img, title):
    img_with_title = np.zeros((height + 30, width, 3), dtype=np.uint8)
    img_with_title[:height, :width] = img
    cv2.putText(img_with_title, title, (10, height + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    return img_with_title

# Thêm tên dưới mỗi ảnh
img1 = add_title(resized_image, titles[0])
img2 = add_title(bright_img, titles[1])
img3 = add_title(dark_img, titles[2])
img4 = add_title(flipped_img, titles[3])
img5 = add_title(blurred_img, titles[4])
img6 = add_title(noisy_img, titles[5])

# Cập nhật chiều cao của các ảnh
height += 30

# Ghép ảnh lại với nhau thành một lưới 2x3
top_row = np.hstack((img1, img2, img3))
bottom_row = np.hstack((img4, img5, img6))
grid = np.vstack((top_row, bottom_row))

# Hiển thị ảnh
cv2.imshow('Image Grid', grid)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Làm việc với thư mục

#### Lời giải bài tập 3

In [ ]:
def process_image(input_image_path, output_folder_path):
    img = cv2.imread(input_image_path)
    if img is None:
        print(f"Không thể đọc ảnh từ đường dẫn: {input_image_path}")
        return
    
    base_filename = os.path.basename(input_image_path)
    filename, ext = os.path.splitext(base_filename)
    
    # Resize
    resized_img = resize_image(img, 320, 240)
    resized_path = os.path.join(output_folder_path, f"{filename}_resized{ext}")
    cv2.imwrite(resized_path, resized_img)
    
    # Tăng sáng
    bright_img = adjust_brightness(resized_img, 50)
    bright_path = os.path.join(output_folder_path, f"{filename}_bright{ext}")
    cv2.imwrite(bright_path, bright_img)

    # Giảm sáng
    dark_img = adjust_brightness(resized_img, -50)
    dark_path = os.path.join(output_folder_path, f"{filename}_dark{ext}")
    cv2.imwrite(dark_path, dark_img)

    # Lật ảnh ngang
    flipped_img = flip_horizontal(resized_img)
    flipped_path = os.path.join(output_folder_path, f"{filename}_flipped{ext}")
    cv2.imwrite(flipped_path, flipped_img)

    # Làm mờ
    blurred_img = blur_image(resized_img)
    blurred_path = os.path.join(output_folder_path, f"{filename}_blurred{ext}")
    cv2.imwrite(blurred_path, blurred_img)

    # Thêm nhiễu
    noisy_img = add_noise(resized_img)
    noisy_path = os.path.join(output_folder_path, f"{filename}_noisy{ext}")
    cv2.imwrite(noisy_path, noisy_img)

In [ ]:
def process_images_in_folder(input_folder_path, output_folder_path):
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for filename in os.listdir(input_folder_path):
        input_image_path = os.path.join(input_folder_path, filename)
        process_image(input_image_path, output_folder_path)

In [ ]:
# Đường dẫn đến thư mục chứa các file ảnh đầu vào và đầu ra
input_folder_path = [...]
output_folder_path = [...]

# Chạy hàm xử lý
process_images_in_folder(input_folder_path, output_folder_path)

### Gán nhãn data

#### Lời giải bài tập 4

In [ ]:
def label_data(input_folder_path, output_folder_path):
    count = 1
    for filename in os.listdir(input_folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_image_path = os.path.join(input_folder_path, filename)
            img = cv2.imread(input_image_path)
            if img is None:
                print(f"Không thể đọc ảnh từ đường dẫn: {input_image_path}")
                return
            new_path = os.path.join(output_folder_path,  "red" + str(count) + ".jpg")
            cv2.imwrite(new_path, img)
        count = count + 1

In [ ]:
input_path = [...]
output_path = [...]

label_data(input_path, output_path)

### Xây dựng bộ data

Kết quả cuối cùng của việc label data sẽ gồm ảnh và một file txt đi kèm chứa thông tin về class, vị trí của vật thể trong ảnh. Sau đó, chúng ta sẽ cần chia data làm 3 thư mục là train, valid và test (thường sẽ theo tỉ lệ 7:2:1).

Để thuận tiện trong việc label và phân chia dữ liệu, hãy truy cập trang web https://roboflow.com/ để có thể tạo dựng bộ data của riêng mình.